# Analiza dnevnih migracij v Sloveniji

V nalogi bom analiziral migracije med Slovenskimi občinami in ugotavljal v kakšnem razmerju so cene nepremičnin ter  delovne razmere in ugotavljal kje bi bilo mogoče izboljšat zaslužek in standard življenja.

Na začetku naložimo vse podatke, ki jih bomo pri analizi uporabili.

In [219]:
import pandas as pd

# Nastavitve prikaza
pd.options.display.max_rows = 20

%matplotlib inline

# Podatki
import os.path

pot_do_podatkov = "./podatki"

nepremicnine = pd.read_csv(os.path.join(pot_do_podatkov, "nepremicnine.csv"), index_col="id")
podjetja = pd.read_csv(os.path.join(pot_do_podatkov, "podjetja.csv"))
razdalje = pd.read_csv(os.path.join(pot_do_podatkov, "razdalje.csv"), index_col="zacetek")
migracije_med_obcinami = pd.read_csv(os.path.join(pot_do_podatkov, "migracije.csv"))

obcine = pd.read_csv(os.path.join(pot_do_podatkov, "obcine.csv"), index_col="ime")

,konec,razdalja,cas
zacetek,,,
Ajdovščina,Ankaran/Ancarano,63510,2470
Ajdovščina,Apače,244477,8953
Ajdovščina,Beltinci,265535,9174
Ajdovščina,Bistrica ob Sotli,204522,7557
Ajdovščina,Bled,131427,4684
...,...,...,...
Žužemberk,Žalec,108281,4509
Žužemberk,Železniki,90346,4747
Žužemberk,Žetale,195613,7236


Poglejmo najprej s kakšnimi podatki bomo delali. Naložili smo podatke o nepremičninah, ki vsebujejo podatke o vrsti nepremičnine, ceni, vrsti posredovanja, številu sob in podobno. Prav tako sem zbral podatke o razdaljah med posameznimi občinami in migracijah, ki nam bodo pomagale ocenit koliko povprečno posameznik izgubi časa in denarja zaradi potrebnih poti.

Poleg teh podatkov, imamo še informacije o večjih Slovenskih podjetjih, kar nam bo pomagalo ocenit pri katera podjetja delajo največjo izgubo s trenutno ureditvijo.

Vse podatke pa bomo združevali po občinah, zato imamo tudi statistične podatke o posamezni občini.

In [40]:
nepremicnine.info()
podjetja.info()
migracije_med_obcinami.info()
razdalje_med_obcinami.info()
obcine.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24934 entries, o6358608 to o6353469
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   posredovanje   24934 non-null  object 
 1   naslov         24934 non-null  object 
 2   url            24934 non-null  object 
 3   kratek_opis    24934 non-null  object 
 4   vrsta          24933 non-null  object 
 5   regija         24933 non-null  object 
 6   upravna_enota  24933 non-null  object 
 7   obcina         24757 non-null  object 
 8   velikost       24925 non-null  float64
 9   cena           24930 non-null  float64
 10  leto           20022 non-null  float64
 11  sobe           5450 non-null   float64
 12  agencija       24934 non-null  object 
 13  cena_na_m2     24921 non-null  float64
dtypes: float64(5), object(9)
memory usage: 2.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9925 entries, 0 to 9924
Data columns (total 6 columns):
 #   Column            N

## Nepremičnine

Najprej izračunajmo povprečno vrednosti nepremičnin v posamezni občini. Pri tem se osredotočimo na hiše in stanovanja, ki se prodajajo in si zabeležimo še koliko nepremičnin prištejemo k povprečju.

Cene nepremičnin nam bodo pomagale ocenit, če je razlika v ceni nepremičnine, glede na to kam ljudje iz te občine migrirajo, smiselna glede na čas in stroške poti. V idealni situaciji bi živel tam, kjer je cena nepremičnin majhna, pot kratka, zaslužek pa velik.

In [210]:
# Vsem nepremičninam izračunamo ceno kvadratnega metra.
nepremicnine["cena_na_m2"] = nepremicnine.cena / nepremicnine.velikost

# Zajem
nepremicnine_po_obcinah = nepremicnine[
    nepremicnine.vrsta.isin(["Hiša", "Stanovanje"]) & 
    (nepremicnine.posredovanje == "prodaja")
].groupby("obcina")
    
# Obdelaj in priključi podatke k občinam.
obcine["cena_na_m2"] = nepremicnine_po_obcinah["cena_na_m2"].mean()
obcine["stevilo_nepremicnin"] = nepremicnine_po_obcinah.size()

## Migracije

Izračunajmo sedaj kako se premikajo migracije po Slovenskih regijah. Zanima nas, kakšen delež migrantov iz posamezne regije gre povprečno v dano drugo regijo. Povedano drugače, zanima nas kakšna je verjetnost, da oseba, ki živi v Ljubljani in migrira, migrira v Maribor, Koper...

To bomo izračunali na sledeč način:

1. Najprej bomo izračunali absolutno število ljudi, ki vsak dan migrira v Ljubljano. Ta podatek dobimo kot razliko delavno aktivnih prebivalcev, ki delajo v Ljubljani (po delovnem mestu) in delavno aktivnih prebivalcev, ki živijo v Ljubljani (po prebivališču), pomnožen z deležem lokalnih delavcev.

1. Glede na izbrano občino bomo izračunali kakšna je verjetnost, da gre ta oseba v druge občine. To bomo določili kot relativno število migracije v posamezno občino glede na vse ostale občine razen te, iz katere oseba prihaja. Ker ljudje raje delajo bližje doma, bom vrednosti obtežil glede na oddaljenost od dane občine.


In [300]:
migracije_po_obcinah = migracije_med_obcinami.groupby('obcina').mean()

# Dodaj podatke v občine.
obcine["delez_migrantov"] = migracije_po_obcinah.delez_migrantov
obcine["delez_lokalnih_delavcev"] = migracije_po_obcinah.delez_lokalnih_delavcev

# Koliko ljudi pride vsak dan v to občino iz druih Slovenskih občin.
obcine["st_migrantov_v_obcino"] = obcine.st_delovno_aktivnih_po_delovnem_mestu - (obcine.st_delovno_aktivnih_po_prebivaliscu * (obcine.delez_lokalnih_delavcev / 100))

# Za vsako občino dobimo podatek o tem koliko časa (sekund) in kakšno razdaljo (metrov)
# potrebuje oseba za pot do vseh ostalih občin.
def razdalja_in_cas_iz_obcine(obcina):
    return razdalje[
        (razdalje.index == obcina) &
        # nekateri podatki imajo vrednost 1000km, zato jih filtriramo.
        (razdalje.razdalja < 500_000)
    ].set_index("konec")

# Kakšna je verjetnost, da migrira nekdo iz dane občine v drugo občino.
def doloci_migracije(obcina):
    # Odstranimo občino v kateri smo.
    ostale_obcine = obcine[obcine.index != obcina].st_migrantov_v_obcino
    
    # Verjetnost, da se prebivalec naše občine odpelje v drugo občino.
    verjetnost = ostale_obcine / ostale_obcine.sum()
    
    # Občine otežimo glede na razdaljo od naše občine.
    oddaljenost = razdalja_in_cas_iz_obcine(obcina).razdalja
    utezi = (1 - pow((oddaljenost / oddaljenost.max()), 0.5))
    
    utezena_verjetnost = verjetnost * utezi

    return utezena_verjetnost / utezena_verjetnost.sum()
    
# Uporabimo narejeno funkcijo na vsaki občini, tako da dobimo razpredelnico o tem, 
# kakšna je verjetnost, da gre nekdo iz poljubne občine v poljubno občino.
migracije = pd.Series(
    obcine.index, 
    name="obcine", 
    index=obcine.index
).apply(
    doloci_migracije
).transpose()

# Izračunamo koliko časa in kilometrov v povprečju posameznik iz dane občine porabi.
def pot(obcina):
    migracije_obcina = migracije[obcina]

    df = pd.DataFrame({ 
        "razdalja": razdalja_in_cas_iz_obcine(obcina).razdalja, 
        "cas": razdalja_in_cas_iz_obcine(obcina).cas, 
        "migracije": migracije_obcina
    }, index=migracije_obcina.index)

    df["rel_razdalja"] = df.razdalja * df.migracije
    df["rel_cas"] = df.cas * df.migracije
    
    return pd.Series(
        [
            df["rel_razdalja"].sum(), 
            df["rel_cas"].sum()
        ], 
        index=["razdalja", "cas"]
    )


# Izračunaj na vseh občinah.
poti = pd.DataFrame(
    pd.Series(obcine.index, index=obcine.index).apply(pot),
    columns=["razdalja", "cas"]
).transpose()

# poti.sort_values("razdalja").tail(20)
# poti["Grosuplje"], poti["Maribor"]
migracije["Grosuplje"].sort_values(ascending=False)

Ljubljana     0.389905
Maribor       0.047547
Celje         0.033890
Novo mesto    0.032765
Kranj         0.030554
                ...   
Muta               NaN
Naklo              NaN
Slovenija          NaN
Tabor              NaN
Štore              NaN
Name: Grosuplje, Length: 213, dtype: float64